# Laser pulse propagation in optical fiber

This notebook shows a simple, scalar implementation of the [Split-Step Fourier Method](https://en.wikipedia.org/wiki/Split-step_method) for solving the [Nonlinear Schrödinger Equation](https://en.wikipedia.org/wiki/Nonlinear_Schrödinger_equation).

$\frac{\partial A}{\partial z}=-\frac{\alpha}{2}A +i \frac{\beta_2}{2} \frac{\partial^2 A}{\partial t^2} + \frac{\beta_3}{6} \frac{\partial^3 A}{\partial t^3} -i \gamma[(|A|^2A) -\frac{i}{\omega_0}\frac{\partial}{\partial t}(|A|^2A) -T_R A\frac{\partial}{\partial t}(|A|^2)]$

This nonlinear partial differential equation models how the envelope and phase of light pulse changes when propagating through a single mode optical fiber, when taking power attenuation ($\alpha$), group velocity dispersion ($\beta_2$), third order dispersion ($\beta_3$), waveguide nonlinearity ($\gamma$) causing self-phase modulation (SPM), self-steepening (SS) and Stimulated Raman-response (SRS) ($T_R$) into account. Note that the effects of SS and and SRS are not included (yet) in the code.

## References

    Ole Krarup. (2024). NLSE-vector-solver. Retrieved from https://github.com/OleKrarup123/NLSE-vector-solver
    Ole Krarup. (2022). The Nonlinear Schrödinger Equation solved in python!. Retrieved from https://colab.research.google.com/drive/1XyLYqrohf5GL6iFSrS6VlHoj_eSm-kAG?usp=sharing

## Importing the modules

In [1]:
from libraries import *
from variables import *
from functions import *
from classes import *

## Initialize class about simulation parameters

In [ ]:
sim_config = SIM_config(N,dt,frequency0,correction_factor,wavelength0)

## Testing the defined functions

In [ ]:
# Initialize Gaussian pulse in the time domain
testPulse = GaussianPulseTime(sim_config.t,amplitude,duration)

fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
plt.title("Initial pulse")
ax1.plot(sim_config.t*1e15,getPower(testPulse)/np.max(getPower(testPulse)),'g-')
ax2.plot(sim_config.t*1e15,abs(testPulse)/np.max(abs(testPulse)),'b-')
ax1.set_xlabel("Time [fs]")
ax1.set_ylabel("Power [a.u.]", color='g')
ax2.set_ylabel("Amplitude [a.u.]", color='b')
ax1.set_ylim(0,1)
ax2.set_ylim(0,1)
plt.xlim(-5*duration*1e15,5*duration*1e15)
savePlot('initial pulse')
plt.show()

## Testing the defined spectrum functions

In [ ]:
# Initialize Gaussian pulse in the frequency domain
testSpectrumTheory = GaussianPulseFrequency(sim_config.f_abs,amplitude,duration)
# Getting spectrum from pulse
testSpectrum=getSpectrumFromPulse(sim_config.t,sim_config.f,testPulse)
# Getting Gaussian spectrum
testSpectrum_2=GaussianSpectrum(sim_config.t,sim_config.f_abs,amplitude,1/duration)
plt.figure()
plt.title("Spectrum of initial pulse")
plt.plot(sim_config.f/1e15,getPower(testSpectrumTheory)/np.max(getPower(testSpectrum)),label="Spectrum of the theory")
plt.plot(sim_config.f/1e15,getPower(testSpectrum)/np.max(getPower(testSpectrum)),label="Spectrum of testPulse")
plt.plot(sim_config.f/1e15,getPower(testSpectrum_2)/np.max(getPower(testSpectrum)),label="Spectrum generated with GaussianSpectrum")
plt.axis([-1/duration*1e-15,1/duration*1e-15,0,1])
plt.xlabel("Frequency [PHz]")
plt.ylabel("Power spectral density [a.u.]")
plt.legend()
savePlot('spectrum of the initial pulse')
plt.show()

## Testing the FWHM function

In [ ]:
FWHM_frequency=FWHM(sim_config.f,getPower(testSpectrum))
FWHM_time=FWHM(sim_config.t,getPower(testPulse))
product=FWHM_frequency*FWHM_time
print(f"The product of the pulse and the spectrum's FWHM is {product}")

## Initialize class about fiber

In [6]:
fiber=Fiber_config(nsteps,Length,gamma,beta2,beta3,alpha_dB_per_m,self_steepening)

# Run SSFM

In [7]:
pulseMatrix, spectrumMatrix = SSFM(fiber,sim_config,testPulse)
testPulse0=np.copy(pulseMatrix[0,:])
testSpectrum0=np.copy(spectrumMatrix[0,:])
testPulseFinal=np.copy(pulseMatrix[-1,:])
testSpectrumFinal=np.copy(spectrumMatrix[-1,:])

## Initialize functions for plotting the results

In [8]:
#%%script false
FWHM_frequency_final=FWHM(sim_config.f,getPower(testSpectrumFinal))
#plotFirstAndLastPulse(pulseMatrix,sim_config)
#plotPulseMatrix2D(pulseMatrix,fiber,sim_config)
#plotFirstAndLastSpectrum(spectrumMatrix,sim_config,FWHM_frequency_final)
#PSD_wavelength(spectrumMatrix,sim_config)
#plotSpectrumMatrix2D_1(spectrumMatrix,fiber,sim_config,FWHM_frequency_final)
#plot_spectrogram(sim_config, pulseMatrix[0,:], 200, 200, dt, label='initial')
#plot_spectrogram(sim_config, pulseMatrix[-1,:], 5000, 5000, dt, label='last')

We get interesting pulse and spectrum shapes out, but are we sure that loss, dispersion and nonlinearity have been applied correctly?

## Comparing the loss effect calculated with SSFM versus theory

In [ ]:
%%script false

#Define new fiber with only loss
fiber_loss_only=Fiber_config(fiber.nsteps, fiber.Length, 0, 0, 0, fiber.alpha_dB_per_m,False)

#Apply theoretical effect of loss only
finalPulse_loss_only_theoretical = np.copy(testPulse0)*np.exp(-fiber.alpha_Np_per_m/2*fiber.Length)

#Calculate effect of loss only numerically
pulseMatrix_loss_only_SSFM,spectrumMatrix_loss_only_SSFM = SSFM(fiber_loss_only,sim_config,testPulse)

finalPulse_loss_only_SSFM=pulseMatrix_loss_only_SSFM[-1,:]

#Do the orange and green curves match?
plt.figure()
plt.title("Initial pulse and final pulse for loss only")
#t=sim_config.t[int(sim_config.number_of_points/2-nrange_time):int(sim_config.number_of_points/2+nrange_time)]*1e15
plt.plot(sim_config.t*1e15,getPower(testPulse0)/np.max(getPower(testPulse0)),label="Initial Pulse")
plt.plot(sim_config.t*1e15,getPower(finalPulse_loss_only_theoretical)/np.max(getPower(testPulse0)),label="Final Pulse theoretical")
plt.plot(sim_config.t*1e15,getPower(finalPulse_loss_only_SSFM)/np.max(getPower(testPulse0)),label="Final Pulse SSFM")
plt.axis([-2*duration*1e15,2*duration*1e15,0,1])
plt.xlabel("Time [fs]")
plt.ylabel("Power [a.u.]")
plt.legend()
savePlot('initial pulse and final pulse for loss only')
plt.show()

#Quantify error by looking at the energy of the difference between numerical and theoretical result. 
loss_err = getEnergy(sim_config.t,(finalPulse_loss_only_SSFM-finalPulse_loss_only_theoretical))/getEnergy(sim_config.t, finalPulse_loss_only_theoretical)
print(f"Loss error computed by energy of difference between theory and SSFM is {loss_err}")

#Alternatively, compute the difference of their two energies
loss_err2 = (getEnergy(sim_config.t,finalPulse_loss_only_SSFM)-getEnergy(sim_config.t,finalPulse_loss_only_theoretical))/getEnergy(sim_config.t, finalPulse_loss_only_theoretical)
print(f"Loss error computed by difference of energy between theory and SSFM is {loss_err2}")

## Comparing the GVD calculated with SSFM versus theory

In [ ]:
%%script false

#Define new fiber with only group velocity dispersion
fiber_disp_only=Fiber_config(fiber.nsteps, fiber.Length, 0, fiber.beta2, 0, 0, False)

#Apply theoretical effect of only group velocity dispersion
disp_theo = np.exp(1j*fiber_disp_only.beta2/2*(2*pi*sim_config.f)**2*fiber_disp_only.Length )  
finalPulse_disp_only_theo = getPulseFromSpectrum(sim_config.t,sim_config.f, getSpectrumFromPulse(sim_config.t,sim_config.f,testPulse0)*disp_theo)

#Calculate group velocity disperson only effect numerically
pulseMatrix_disp_only,_ = SSFM(fiber_disp_only,sim_config,testPulse0)

finalPulse_disp_only_SSFM=pulseMatrix_disp_only[-1,:]

#Do the orange and green curves match?
plt.figure()
plt.title("Initial pulse and final pulse for GVD only")
plt.plot(sim_config.t*1e15,getPower(testPulse0)/np.max(getPower(testPulse0)),label="Initial Pulse")
plt.plot(sim_config.t*1e15,getPower(finalPulse_disp_only_theo)/np.max(getPower(testPulse0)),label="Final Pulse theoretical")
plt.plot(sim_config.t*1e15,getPower(finalPulse_disp_only_SSFM)/np.max(getPower(testPulse0)),label="Final Pulse SSFM")
plt.axis([-5*duration*1e15,5*duration*1e15,0,1])
plt.xlabel("Time [fs]")
plt.ylabel("Power [a.u.]")
plt.legend()
savePlot('initial and final pulse for GVD only')
plt.show()

#Quantify error by looking at the energy of the difference between numerical and theoretical result. 
disp_err = getEnergy(sim_config.t,(finalPulse_disp_only_SSFM-finalPulse_disp_only_theo))/getEnergy(sim_config.t, finalPulse_disp_only_theo)
print(f"Disp error computed by energy of difference between theory and SSFM is {disp_err}")

#Alternatively, compute the difference of their two energies
disp_err2 = (getEnergy(sim_config.t,finalPulse_disp_only_SSFM)-getEnergy(sim_config.t,finalPulse_disp_only_theo))/getEnergy(sim_config.t, finalPulse_disp_only_theo)
print(f"Disp error computed by difference of energy between theory and SSFM is {disp_err2}")

Positive dispersion (beta2>0) should lead to a negative (red) frequency shift (chirp) for the leading part of the pulse. Let's check that this is correct.

In [ ]:
%%script false

#Is the chirp negative (red) in the front and positive (blue) in the back for beta2>0? 
#Is the chirp positive (blue) in the front and negative (red) in the back for beta2<0? 

plt.figure()
plt.title(f"Initial chirp and final chirp for disp only with beta2 = {fiber.beta2*1e30} fs^2/m")
plt.plot(sim_config.t*1e15,getChirp(sim_config.t,testPulse0)/1e15,label="Initial Pulse")
plt.plot(sim_config.t*1e15,getChirp(sim_config.t,finalPulse_disp_only_theo)/1e15,label="Final Pulse theoretical")
plt.plot(sim_config.t*1e15,getChirp(sim_config.t,finalPulse_disp_only_SSFM)/1e15,label="Final Pulse SSFM")
#plt.axis([-10*duration*1e15,10*duration*1e15,-1e-1,1e-1])
plt.xlabel("Time [fs]")
plt.ylabel("Chirp [PHz]")
plt.legend()
plt.show() 

## Comparing the TOD calculated with SSFM versus theory

In [ ]:
%%script false

#Define new fiber with only third order dispersion
fiber_disp_only=Fiber_config(fiber.nsteps, fiber.Length, 0, 0, fiber.beta3, 0, False)

#Apply theoretical effect of only third order dispersion
disp_theo = np.exp(1j*fiber_disp_only.beta3/6*(2*pi*sim_config.f)**3*fiber_disp_only.Length )  
finalPulse_disp_only_theo = getPulseFromSpectrum(sim_config.t,sim_config.f, getSpectrumFromPulse(sim_config.t,sim_config.f,testPulse0)*disp_theo)

#Calculate third order disperson only effect numerically
pulseMatrix_disp_only,_ = SSFM(fiber_disp_only,sim_config,testPulse0)

finalPulse_disp_only_SSFM=pulseMatrix_disp_only[-1,:]

#Do the orange and green curves match?
plt.figure()
plt.title("Initial pulse and final pulse for TOD only")
plt.plot(sim_config.t*1e15,getPower(testPulse0)/np.max(getPower(testPulse0)),label="Initial Pulse")
plt.plot(sim_config.t*1e15,getPower(finalPulse_disp_only_theo)/np.max(getPower(testPulse0)),label="Final Pulse theoretical")
plt.plot(sim_config.t*1e15,getPower(finalPulse_disp_only_SSFM)/np.max(getPower(testPulse0)),label="Final Pulse SSFM")
plt.axis([-10*duration*1e15,10*duration*1e15,0,1])
plt.xlabel("Time [fs]")
plt.ylabel("Power [a.u.]")
plt.legend()
savePlot('initial pulse and final pulse for TOD only')
plt.show()

#Quantify error by looking at the energy of the difference between numerical and theoretical result. 
disp_err = getEnergy(sim_config.t,(finalPulse_disp_only_SSFM-finalPulse_disp_only_theo))/getEnergy(sim_config.t, finalPulse_disp_only_theo)
print(f"Disp error computed by energy of difference between theory and SSFM is {disp_err}")

#Alternatively, compute the difference of their two energies
disp_err2 = (getEnergy(sim_config.t,finalPulse_disp_only_SSFM)-getEnergy(sim_config.t,finalPulse_disp_only_theo))/getEnergy(sim_config.t, finalPulse_disp_only_theo)
print(f"Disp error computed by difference of energy between theory and SSFM is {disp_err2}")

## Comparing the $\gamma$ effect calculated with SSFM versus theory

In [ ]:
%%script false

#Define new fiber with only gamma
fiber_gamma_only=Fiber_config(fiber.nsteps, fiber.Length, fiber.gamma, 0, 0, 0, False)

#Apply theoretical effect of only gamma
finalPulse_gamma_only_theoretical = np.copy(testPulse0)*np.exp(1j*(fiber_gamma_only.Length)*(fiber_gamma_only.gamma)*getPower(testPulse0))

finalSpectrum_gamma_only_theoretical=getSpectrumFromPulse(sim_config.t,sim_config.f_abs_plot,finalPulse_gamma_only_theoretical)

#Calculate gamma only effect numerically
pulseMatrix_gamma_only_SSFM,spectrumMatrix_gamma_only_SSFM = SSFM(fiber_gamma_only,sim_config,testPulse0)

finalSpectrum_gamma_only_SSFM = spectrumMatrix_gamma_only_SSFM[-1,:]

FWHM_frequency_final_gamma_only_SSFM=FWHM(sim_config.f_abs_plot,getPower(finalSpectrum_gamma_only_SSFM))

#Do the orange and green curves match?
plt.figure()
plt.title("Initial spectrum and final spectrum for gamma only")
plt.plot(sim_config.f_abs_plot/1e15,getPower(testSpectrum0)/np.max(getPower(testSpectrum0)),label="Initial Spectrum")
plt.plot(sim_config.f_abs_plot/1e15,getPower(finalSpectrum_gamma_only_theoretical)/np.max(getPower(testSpectrum0)),label="Final Spectrum theoretical")
plt.plot(sim_config.f_abs_plot/1e15,getPower(finalSpectrum_gamma_only_SSFM)/np.max(getPower(testSpectrum0)),label="Final Spectrum SSFM")
plt.axis([0,2*sim_config.frequency0_without_correction/1e15,0,1])
#plt.axis([-1/(duration*1e15)*5,1/(duration*1e15)*5,0,1])
plt.xlabel("Frequency [PHZ]")
plt.ylabel("Power spectrum density [a.u.]")
plt.legend()
savePlot('initial spectrum and final spectrum for gamma only')
plt.show()

#Quantify error by looking at the energy of the difference between numerical and theoretical result. 
gamma_err = getEnergy(sim_config.f_abs_plot,(finalSpectrum_gamma_only_SSFM-finalSpectrum_gamma_only_theoretical))/getEnergy(sim_config.f_abs_plot, finalSpectrum_gamma_only_theoretical)
print(f"Gamma error computed by energy of difference between theory and SSFM is {gamma_err}")

#Alternatively, compute the difference of their two energies
gamma_err2 = (getEnergy(sim_config.f_abs_plot,finalSpectrum_gamma_only_SSFM)-getEnergy(sim_config.f_abs_plot,finalSpectrum_gamma_only_theoretical))/getEnergy(sim_config.f_abs_plot, finalSpectrum_gamma_only_theoretical)
print(f"Gamma error computed by difference of energy between theory and SSFM is {gamma_err2}")

## Checking if self-phase modulation causes a negative (red) leading chirp

In [ ]:
%%script false

#Is the chirp negative (red) in the front and positive (blue) in the back for gamma>0? 
#Is the chirp positive (blue) in the front and negative (red) in the back for gamma<0? 
plt.figure()
plt.title(f"Initial chirp and final chirp for gamma only with gamma = {fiber_gamma_only.gamma} 1/W*m")
plt.plot(sim_config.t*1e15,getChirp(sim_config.t,testPulse0)/1e15,label="Initial Pulse")
plt.plot(sim_config.t*1e15,getChirp(sim_config.t,finalPulse_gamma_only_theoretical)/1e15,label="Final Pulse theoretical")
plt.plot(sim_config.t*1e15,getChirp(sim_config.t,finalPulse_gamma_only_theoretical)/1e15,label="Final Pulse SSFM")
#plt.axis([-150,150,-1,1])
plt.xlabel("Time [fs]")
plt.ylabel("Chirp [PHz]")
plt.legend()
plt.show() 

## Checking if we got a shorter (dispersion-compensated) pulse FWHM than the initial pulse FWHM

In [ ]:
#%%script false

time=sim_config.t
testSpectrumFinalAbsoluteValue=abs(testSpectrumFinal)
pulseFromTestSpectrumFinal=ifftshift(ifft(testSpectrumFinalAbsoluteValue))/dt
pulseFromTestSpectrumFinalAbsoluteValue=abs(pulseFromTestSpectrumFinal)
testPulseAbsoluteValue=abs(testPulse)

plt.figure()
plt.title("Initial and dispersion-compensated final pulse")
plt.plot(time*1e15,getPower(testPulseAbsoluteValue)/np.max(getPower(pulseFromTestSpectrumFinalAbsoluteValue)),label="Initial pulse")
plt.plot(time*1e15,getPower(pulseFromTestSpectrumFinalAbsoluteValue)/np.max(getPower(pulseFromTestSpectrumFinalAbsoluteValue)),label="Final pulse")
plt.axis([-duration*1e15,duration*1e15,0,1])
plt.xlabel("Time [fs]")
plt.ylabel("Power [a.u.]")
plt.legend()
savePlot('initial and dispersion-compensated final pulse')
plt.show()

print(f"The initial pulse's FWHM is {FWHM(time,getPower(testPulseAbsoluteValue))} s")
print(f"The final pulse's FWHM is {FWHM(time,getPower(pulseFromTestSpectrumFinalAbsoluteValue))} s")

## Comparing the self-steepening effect versus without it

In [ ]:
%%script false

#Define new fiber without self-steepening
fiber_without_self_steepening=Fiber_config(fiber.nsteps, fiber.Length, fiber.gamma, fiber.beta2, fiber.beta3, fiber.alpha_dB_per_m, False)

#Calculate pulse propagation numerically without self-steepening
pulseMatrix_without_self_steepening_SSFM,_=SSFM(fiber_without_self_steepening,sim_config,testPulse0)

#Final pulse
finalPulse_without_self_steepening_SSFM=pulseMatrix_without_self_steepening_SSFM[-1,:]

#Define new fiber with self-steepening
fiber_with_self_steepening=Fiber_config(fiber.nsteps, fiber.Length, fiber.gamma, fiber.beta2, fiber.beta3, fiber.alpha_dB_per_m, True)

#Calculate pulse propagation numerically with self-steepening
pulseMatrix_with_self_steepening_SSFM,_=SSFM(fiber_with_self_steepening,sim_config,testPulse0)

#Final pulse
finalPulse_with_self_steepening_SSFM=pulseMatrix_with_self_steepening_SSFM[-1,:]

time=sim_config.t
plt.figure()
plt.title("With and without self-steepening")
plt.plot(time*1e15,getPower(finalPulse_without_self_steepening_SSFM)/np.max(getPower(finalPulse_without_self_steepening_SSFM)),label="Without self-steepening")
plt.plot(time*1e15,getPower(finalPulse_with_self_steepening_SSFM)/np.max(getPower(finalPulse_with_self_steepening_SSFM)),label="With self-steepening")
plt.xlabel("Time [fs]")
plt.ylabel("Power [a.u.]")
plt.legend()
savePlot('With and without self-steepening')
plt.show()

## Checking how reducing the number of steps affects pulse accuracy

According to [theoretical considerations](https://en.wikipedia.org/wiki/Split-step_method), the error should decrease according to (Δz)^2. Let's check if this is the case by

1.  Using the pulse computed with nsteps as a baseline.
2.  Running SSFM with increasing step numbers (=decreasing step size) and determining the error between the calculated pulse and the baseline.
3.  Plotting the error as a function of nsteps, fitting a model like "b*(nsteps)^a" to the data and checking if a=-2.

In [ ]:
%%script false

#First, make a copy of the pulse computed with the original fiber settings and nsteps
baselinePulse = np.copy(testPulseFinal)

#Make array of the number of steps
#nsteps_list=np.logspace(0,np.log10(fiber.nsteps),9)  
nsteps_list=[2**4, 2**5, 2**6, 2**7, 2**8, 2**9, 2**10]
      
#Pre-define arrays for holding errors
err_list=np.zeros_like(nsteps_list)*1.0

#Start figure and plot the pulse with the large number of steps
plt.figure()
plt.plot(sim_config.t*1e15,getPower(baselinePulse)/1e12,label=f"Baseline for nsteps = {fiber.nsteps}")

#For each iteration, define a new fiber with a different number of steps but the same physical properties
for i in range(0,len(nsteps_list)):
  nsteps_i=int(nsteps_list[i])
  fiber_i=Fiber_config(nsteps_i, fiber.Length, fiber.gamma, fiber.beta2, fiber.beta3, fiber.alpha_dB_per_m, False)
  
  #Run the simulation and plot the computed pulse
  pulseMatrix_i,_=SSFM(fiber_i,sim_config,np.copy(testPulse0))
  
  pulse_i=pulseMatrix_i[-1,:]
  
  plt.plot(sim_config.t*1e15,getPower(pulse_i)/1e12,label=f"Pulse for nsteps = {nsteps_i}")
  
  #Compute and store errors
  err = getEnergy(sim_config.t,(pulse_i-baselinePulse))/getEnergy(sim_config.t, baselinePulse)*100
  err_list[i]=err

plt.xlabel("Time [fs]")
plt.ylabel("Power [TW]")
plt.legend(bbox_to_anchor=(1.05,0.3))
plt.show()

## Convergence of the solution

Now make a plot of the error versus nsteps

In [ ]:
%%script false

x=np.log10(nsteps_list[1:-1])
y=np.log10(err_list[1:-1]/np.max(err_list))

model = np.polyfit(x, y, 1)
print("Model parameters [a,b]=",model)
y_model=np.poly1d(model)
plt.figure()
plt.plot(x,y_model(x),'r-',label=f"$err(1 step)*nsteps^{  {np.round(model[0],3)}}$")
plt.plot(x,y,'.')
plt.xlabel('log10(nsteps)')
plt.ylabel('log10(err/err_max)')
plt.legend()
plt.show()

## Calculating the absorption coefficient from Sellmeier formula using Kramers-Kronig relation

In [ ]:
%%script false

x = np.arange(frequency_lower,frequency_upper,1e10) # Frequency step array
#x = sim_config.f
y_1 = getRefractiveIndex(x,B1,B2,B3,C1,C2,C3)
y_2 = getExtintionCoefficient(y_1)
y_3 = getAbsorptionCoefficient(y_2,x)

plotRefractiveIndex(x,y_1)
plotExtintionCoefficient(x,y_2)
plotAbsorptionCoefficient(x,y_3)

## Calculating the GVD from Sellmeier formula

In [ ]:
%%script false

y_4 = getFirstDerivateveOfRefractiveIndex(x,y_1,B1,B2,B3,C1,C2,C3)
y_5 = getSecondDerivateveOfRefractiveIndex(x,y_1,y_4,B1,B2,B3,C1,C2,C3)
y_6 = getGroupVelocityDispersion(x,y_5)

plotFirstDerivateveOfRefractiveIndex(x,y_4)
plotSecondDerivateveOfRefractiveIndex(x,y_5)
plotGroupVelocityDispersion(x,y_6)